# Statevectors

This notebook compares statevectors of random circuits using JMStateVectorSim and Qiskit.

In [1]:
import numpy as np

from JQCLib.Utils.Utilities import fromRaw
from JQCLib.Classes.JMStateVectorSimClass import JMStateVectorSim, FeatureEncode
from JQCLib.Classes.QCSamplerClass import QCSampler

#Qiskit stuff. Only needed for comparison
from qiskit import QuantumCircuit
from qiskit import execute
from qiskit.circuit import ParameterVector
from qiskit.providers.aer import QasmSimulator

### Test configuration

In [2]:
#Possible gates
NUMGATES = 5    #Identity, Hadamard, Rx, Ry, Rz

#Circuit size
QUBITS = 4
LAYERS = 12

### Feature map for Qiskit. Must match FeatureEncode(...) in JMStateVectorSimClass

In [3]:
def QiskitFeaturemap(no_qubits):

    feature_map = QuantumCircuit(no_qubits)

    #Vector for encoding the features
    ip = ParameterVector('Features',no_qubits) 

    for i in range(no_qubits):
        feature_map.h(i)
        feature_map.rz(ip[i], i)

    return feature_map

### Sample circuits and parameters

In [4]:
def randomcircuit(Qubits, Layers):

    #Largest possible gate in config
    MAXGATES = NUMGATES + Qubits - 1

    #Generate random circuit & wash
    circuit = fromRaw(np.random.randint(0, MAXGATES, (Qubits,Layers)))

    #Count trainable parameters in circuit    
    pcount = np.count_nonzero(np.logical_and(circuit>1, circuit<NUMGATES))

    #Sample inputs uniformly
    inputs = np.random.uniform(-np.pi,np.pi,Qubits).tolist()   

    #Sample weights uniformly
    weights = np.random.uniform(-np.pi,np.pi,pcount).tolist()

    #Return tuple of circuit data and sample weights
    return circuit, inputs, weights

### Wrapper around JMStateVectorSim to get statevector

In [5]:
def CustomStatevector(circuit, input,  weights):    
    return JMStateVectorSim(circuit,True).Statevector(weights, FeatureEncode(input))                                      

### Wrapper around Qiskit code to get statevector

In [6]:
def CTQiskit(circuit, input, weights, verbose = False):

    #Create QSampler from test data
    QS = QCSampler(circuit)            #<- Let's use the QCSampler to handle circuit creation with Qiskit

    #The circuit
    qc = QuantumCircuit(QUBITS)

    qc.compose(qiskitff.bind_parameters(input), range(QUBITS), inplace=True)
    qc.compose(QS.Parameterize(weights), range(QUBITS), inplace=True)    

    #Output circuit on screen
    if(verbose):
        #display(qc.draw('mpl',reverse_bits=True))
        display(qc.draw('mpl'))

    #Save the state vector
    qc.save_statevector()

    #Execute & get result    
    job = execute(qc, backend)
    job_result = job.result()

    #Return state vector
    return np.array(job_result.get_statevector(qc)).reshape(-1,1)

### Compare statevectors from Qiskit & JMStateVectorSim

In [7]:
#Qiskit config
backend = QasmSimulator()
qiskitff = QiskitFeaturemap(QUBITS)

#Num tests
TESTS = 10

#Run
for circuit, input, weights in [randomcircuit(QUBITS,LAYERS) for _ in range(TESTS)]:
    qr = CTQiskit(circuit, input, weights)      
    jr = CustomStatevector(circuit, input, weights)
    print("Statevectors matched:",np.all(np.isclose(qr,jr)))    

Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
Statevectors matched: True
